# Personalized Learning Demo - Quickstart

A full-stack A2UI sample demonstrating personalized educational content generation.

**Contributed by Google Public Sector's Rapid Innovation Team.**

![Personalized Learning Demo](assets/hero.png)

---

## The Scenario

**Maria Thompson** is a pre-med student at Cymbal University preparing for the MCAT. She excels in biology (92%) but struggles with chemistry concepts—particularly the common misconception that "energy is stored in ATP bonds."

How we find that information about a student's misconceptions across multiple courses is actually part of a broader project at Google Public Sector. But for now, we're just running with information we have related to one of those misconceptions, represented by the text files in the `learner_context/` directory.

This demo includes a [learner profile visualization](http://localhost:5174/maria-context.html) showing Maria's:
- Academic background and current proficiency levels
- Identified misconceptions to address
- Learning preferences (visual-kinesthetic, sports/gym analogies)

This profile represents the kind of data a real personalization pipeline would generate from learning management systems, assessment results, and curriculum graphs. Once we have that data, how do we best use it to impact a student's learning trajectory?

We think the A2UI framework enables an excellent learning experience, and this demo intends to show how.

---

## How Content Is Generated

**Content Source:** [OpenStax](https://openstax.org/) — free, peer-reviewed textbooks covering 167 chapters across biology, chemistry, physics, and more.

**Generation Pipeline:**
- User requests a topic (e.g., "Help me understand ATP")
- The agent uses an LLM to match the topic to the most relevant OpenStax chapter
- Content is fetched and transformed into A2UI components (flashcards, quizzes)
- The frontend renders whatever A2UI JSON the agent returns

**Learn More:**
- [How A2UI Works](http://localhost:5174/a2ui-primer.html) — interactive explanation in the demo
- [A2UI Specification](../../docs/) — canonical documentation in this repo

---

## What You'll Learn

| Concept | What This Demo Shows |
|---------|---------------------|
| **Remote Agent Deployment** | Deploy an AI agent to Vertex AI Agent Engine that runs independently from your UI |
| **A2A Protocol** | Use the Agent-to-Agent protocol to communicate between your frontend and the remote agent |
| **Custom UI Components** | Extend A2UI with custom components (Flashcard, QuizCard) beyond the standard library |
| **Dynamic Content Generation** | Generate personalized A2UI JSON on-the-fly based on user requests |
| **Dynamic Context from GCS** | Load learner profiles from Cloud Storage — swap context without redeploying |
| **Intelligent Content Matching** | Use LLMs to match user topics to relevant textbook content (167 OpenStax chapters) |

---

## Architecture

![Architecture Diagram](assets/architecture.jpg)

In production agentic systems:
- **Agents run remotely** — they scale independently, can be updated without redeploying the UI, and may be operated by third parties
- **UI is decoupled** — the frontend renders whatever A2UI JSON the agent sends, without knowing the agent's implementation
- **A2A enables interoperability** — any A2A-compatible agent can power your UI, regardless of how it's built
- **Context is dynamic** — learner profiles are loaded from GCS at runtime, enabling personalization without redeployment

---

## How This Notebook Is Organized

| Section | What It Does |
|---------|--------------|
| **Step 1: Environment Setup** | Creates Python virtual environment and installs all dependencies |
| **Step 2: Configuration** | Sets your GCP project ID |
| **Step 3: GCP Authentication** | Authenticates with Google Cloud, enables APIs, and uploads learner context to GCS |
| **Step 4: Deploy Agent** | Deploys the AI agent to Vertex AI Agent Engine |
| **Step 5: Configure & Run** | Creates config files and launches the demo |
| **Step 6 (Optional)** | Generate audio/video content with NotebookLM |
| **Step 7: Production Deployment** | Deploy to Cloud Run + Firebase Hosting for a shareable URL |
| **Appendix: Local Development** | Run entirely locally without cloud deployment |

---

## Prerequisites

- **Node.js 18+** — [Download](https://nodejs.org/)
- **Python 3.11+** — [Download](https://www.python.org/downloads/)
- **Google Cloud project with billing enabled** — [Console](https://console.cloud.google.com/)
- **gcloud CLI installed** — [Install Guide](https://cloud.google.com/sdk/docs/install)

## Imports

Run this cell first to load all Python modules used throughout the notebook.

In [1]:
import subprocess
import sys
import os

## Step 1: Environment Setup

First, we'll create an isolated Python environment and install all dependencies. This ensures the demo doesn't conflict with other Python projects on your system.

### 1a. Create Python Virtual Environment

In [2]:
# Create virtual environment if it doesn't exist
venv_path = os.path.join(os.getcwd(), ".venv")
if not os.path.exists(venv_path):
    print("Creating Python virtual environment...")
    subprocess.run([sys.executable, "-m", "venv", ".venv"], check=True)
    print(f"✅ Created virtual environment at {venv_path}")
else:
    print(f"✅ Virtual environment already exists at {venv_path}")

✅ Virtual environment already exists at /Users/samgoodgame/Desktop/A2UI/samples/personalized_learning/.venv


### 1b. Install Python Dependencies

After selecting the `.venv` kernel, run this cell to install all required Python packages.

**Note:** We explicitly use `https://pypi.org/simple/` to ensure packages come from the official Python Package Index, avoiding issues with corporate proxies or custom registries.

In [3]:
# Install Python dependencies using the canonical PyPI index
print("Installing Python dependencies from PyPI...")
packages = [
    "google-adk>=0.3.0",
    "google-genai>=1.0.0",
    "google-cloud-storage>=2.10.0",
    "python-dotenv>=1.0.0",
    "litellm>=1.0.0",
    "vertexai",
]

subprocess.run([
    sys.executable, "-m", "pip", "install", "-q",
    "--index-url", "https://pypi.org/simple/",
    "--trusted-host", "pypi.org",
    "--trusted-host", "files.pythonhosted.org",
    *packages
], check=True)

print("✅ Python dependencies installed")

Installing Python dependencies from PyPI...
✅ Python dependencies installed



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


### 1c. Install Node.js Dependencies

Now we'll install the frontend dependencies. This includes the A2UI renderer library and the demo's own packages.

In [4]:
# Build the A2UI Lit renderer (using public npm registry)
print("Building A2UI Lit renderer...")
subprocess.run(
    "npm install --registry https://registry.npmjs.org/ && npm run build",
    shell=True,
    cwd="../../renderers/lit",
    check=True
)
print("✅ A2UI renderer built")

# Install demo dependencies
print("\nInstalling demo dependencies...")
subprocess.run(
    "npm install --registry https://registry.npmjs.org/",
    shell=True,
    check=True
)
print("✅ Demo dependencies installed")

Building A2UI Lit renderer...


npm warn Unknown project config "always-auth" (//us-npm.pkg.dev/oss-exit-gate-prod/a2ui--npm/:always-auth). This will stop working in the next major version of npm.
npm warn Unknown global config "always-auth" (//us-npm.pkg.dev/artifact-foundry-prod/ah-3p-staging-npm/:always-auth). This will stop working in the next major version of npm.



up to date, audited 99 packages in 399ms

12 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities

> @a2ui/lit@0.8.1 build
> wireit



npm warn Unknown project config "always-auth" (//us-npm.pkg.dev/oss-exit-gate-prod/a2ui--npm/:always-auth). This will stop working in the next major version of npm.
npm warn Unknown global config "always-auth" (//us-npm.pkg.dev/artifact-foundry-prod/ah-3p-staging-npm/:always-auth). This will stop working in the next major version of npm.
Analyzing


✅ Ran 0 scripts and skipped 2 in 0s.
✅ A2UI renderer built

Installing demo dependencies...


npm warn Unknown global config "always-auth" (//us-npm.pkg.dev/artifact-foundry-prod/ah-3p-staging-npm/:always-auth). This will stop working in the next major version of npm.



added 68 packages, and audited 274 packages in 1s

32 packages are looking for funding
  run `npm fund` for details

12 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
✅ Demo dependencies installed


## Step 2: Configuration

Set your Google Cloud project ID below. This is the project where the agent will be deployed.

In [5]:
PROJECT_ID = "a2ui-test"  # <-- CHANGE THIS to your GCP project ID
LOCATION = "us-central1"  # Agent Engine requires us-central1

## Step 3: GCP Authentication & API Setup

Authenticate with Google Cloud and enable the required APIs. This will open browser windows for authentication.

In [ ]:
# Authenticate with Google Cloud
!gcloud auth login
!gcloud config set project {PROJECT_ID}
!gcloud auth application-default login

In [ ]:
# Enable required APIs
!gcloud services enable aiplatform.googleapis.com
!gcloud services enable cloudbuild.googleapis.com
!gcloud services enable storage.googleapis.com
!gcloud services enable cloudresourcemanager.googleapis.com

# Create staging bucket for Agent Engine (if it doesn't exist)
!gsutil mb -l us-central1 gs://{PROJECT_ID}_cloudbuild 2>/dev/null || echo "Staging bucket already exists"

# Create learner context bucket (for dynamic context loading)
CONTEXT_BUCKET = f"{PROJECT_ID}-learner-context"
!gsutil mb -l us-central1 gs://{CONTEXT_BUCKET} 2>/dev/null || echo "Context bucket already exists"

# Create OpenStax content bucket
OPENSTAX_BUCKET = f"{PROJECT_ID}-openstax"
!gsutil mb -l us-central1 gs://{OPENSTAX_BUCKET} 2>/dev/null || echo "OpenStax bucket already exists"

# Upload learner context files to GCS
print(f"\nUploading learner context files to gs://{CONTEXT_BUCKET}/learner_context/...")
!gsutil -m cp learner_context/*.txt gs://{CONTEXT_BUCKET}/learner_context/

print(f"\n✅ GCP APIs enabled and buckets ready")
print(f"   Staging bucket: gs://{PROJECT_ID}_cloudbuild")
print(f"   Context bucket: gs://{CONTEXT_BUCKET}/learner_context/")
print(f"   OpenStax bucket: gs://{OPENSTAX_BUCKET}/")

### 3b. Download OpenStax Textbook Content (Optional but Recommended)

The agent fetches **actual OpenStax textbook content** to generate accurate flashcards and quizzes. Content can be fetched:

1. **From GitHub (default)** — Works out of the box, but adds latency per request
2. **From GCS (recommended)** — Pre-download all modules for faster responses

Run the cell below to download all 200+ OpenStax Biology modules to your GCS bucket. This takes ~2 minutes but makes the demo much faster.

In [ ]:
# Download OpenStax Biology modules to GCS (takes ~2 minutes)
# This is optional - the agent will fall back to fetching from GitHub if GCS is empty

OPENSTAX_BUCKET = f"{PROJECT_ID}-openstax"

print(f"Downloading OpenStax Biology modules to gs://{OPENSTAX_BUCKET}/openstax_modules/...")
print("This fetches ~200 textbook modules from GitHub and uploads to GCS.\n")

result = subprocess.run(
    [sys.executable, "download_openstax.py", 
     "--bucket", OPENSTAX_BUCKET,
     "--prefix", "openstax_modules/",
     "--workers", "5"],
    cwd="agent"
)

if result.returncode == 0:
    print(f"\n✅ OpenStax content ready at gs://{OPENSTAX_BUCKET}/openstax_modules/")
else:
    print("\n⚠️  Download had issues, but the agent will fall back to GitHub fetching.")

### 3c. Media Assets (Optional)

The demo can display **podcast and video** to show A2UI's flexible modality support. This is purely demonstrative—we're not generating audio/video with A2UI, just showing the UI can render these media types.

**To add media:**
1. Generate audio/video using [NotebookLM](https://notebooklm.google.com/) (upload the `learner_context/` files, then generate Audio/Video Overviews)
2. Place files in `public/assets/` as `podcast.m4a` and `video.mp4`
3. Run the cell below to upload to GCS for production

In [ ]:
# Upload media from public/assets/ to GCS
MEDIA_BUCKET = f"{PROJECT_ID}-media"

!gsutil mb -l us-central1 gs://{MEDIA_BUCKET} 2>/dev/null || echo "Media bucket exists"

for filename in ["podcast.m4a", "video.mp4"]:
    path = f"public/assets/{filename}"
    if os.path.exists(path):
        !gsutil cp {path} gs://{MEDIA_BUCKET}/assets/{filename}
        print(f"✅ Uploaded {filename}")
    else:
        print(f"⚠️  {path} not found - generate with NotebookLM first")

Media bucket exists
Copying file://public/assets/podcast.m4a [Content-Type=audio/mp4a-latm]...


#### How to Generate Media with NotebookLM

1. Go to [notebooklm.google.com](https://notebooklm.google.com/)
2. Create a new notebook and upload the `learner_context/` files with the extension `.txt`
3. Click **Notebook guide** → **Audio Overview** → **Generate** (or **Video Overview** for video)
4. Download the generated file and save to `public/assets/podcast.m4a` or `public/assets/video.mp4`
5. Run the cell above to upload to GCS

## Step 4: Deploy the A2UI Agent

The agent generates personalized learning content and runs on Vertex AI Agent Engine. Deployment takes 2-5 minutes.

**Why deploy remotely?** A2UI is designed for remote agents - your UI runs in the browser while the agent runs on a server. This mirrors real-world architectures where agents scale independently and may even be operated by third parties.

### Dynamic Learner Context

The agent loads learner profile data from **Cloud Storage at runtime**. This means you can:

1. **Switch students instantly** — Replace the files in `gs://{PROJECT_ID}-learner-context/learner_context/` with a different student's profile
2. **Update without redeploying** — Change misconceptions, learning preferences, or curriculum focus without touching the agent
3. **A/B test personalization** — Point different users to different context buckets

**To personalize for a different student:**
```bash
# Edit the learner profile locally
nano learner_context/01_maria_learner_profile.txt

# Upload to GCS (agent picks up changes on next request)
gsutil cp learner_context/*.txt gs://{PROJECT_ID}-learner-context/learner_context/
```

The agent will automatically use the new context for all subsequent requests.

### Performance Note

The agent uses ADK's context caching for conversation history. For production systems with large textbook corpora, consider Gemini's explicit context cache (requires 32k+ tokens) to cache the full OpenStax content across requests.

In [ ]:
# Deploy the agent to Vertex AI Agent Engine (takes 2-5 minutes)
# No wheel needed - the ServerSideAgent class is self-contained
print("Deploying agent to Vertex AI Agent Engine...")
print("This takes 2-5 minutes. Watch for the Resource ID at the end.\n")

# Use the context bucket created in Step 3
CONTEXT_BUCKET = f"{PROJECT_ID}-learner-context"

result = subprocess.run(
    [sys.executable, "deploy.py", 
     "--project", PROJECT_ID, 
     "--location", LOCATION,
     "--context-bucket", CONTEXT_BUCKET]
)

if result.returncode != 0:
    print("\n❌ Deployment failed. Check the error messages above.")
else:
    print("\n✅ Deployment complete! Copy the Resource ID from the output above.")

## Step 5: Configure & Run

Fill in the Resource ID from the deployment output above, then create the configuration file.

In [ ]:
# Get your project NUMBER (different from project ID)
result = subprocess.run(
    ["gcloud", "projects", "describe", PROJECT_ID, "--format=value(projectNumber)"], 
    capture_output=True, text=True
)
PROJECT_NUMBER = result.stdout.strip()
print(f"Project Number: {PROJECT_NUMBER}")

In [ ]:
# Paste the Resource ID from the deployment output in Step 4
AGENT_RESOURCE_ID = "743456777353297920"  # <-- PASTE YOUR RESOURCE ID HERE

In [ ]:
# Create .env file
env_content = f"""# Generated by Quickstart.ipynb
GOOGLE_CLOUD_PROJECT={PROJECT_ID}
AGENT_ENGINE_PROJECT_NUMBER={PROJECT_NUMBER}
AGENT_ENGINE_RESOURCE_ID={AGENT_RESOURCE_ID}
"""

with open(".env", "w") as f:
    f.write(env_content)

print("Created .env file:")
print(env_content)
print("✅ Configuration complete!")

### Run the Demo Locally

Everything is set up! Run these commands in your terminal:

```bash
cd samples/personalized_learning
npm run dev
```

Then open **http://localhost:5174**

### Test Prompts

| Prompt | What Happens |
|--------|-------------|
| "Help me understand ATP" | Flashcards with OpenStax citation |
| "Quiz me on meiosis" | Interactive quiz with citation |
| "Flashcards for cell energy" | LLM matches to ATP chapter |
| "Play the podcast" | Audio player (requires media in Step 3c) |
| "Show me a video" | Video player (requires media in Step 3c) |

---

## Step 7: Production Deployment

Deploy to Cloud Run for a shareable URL with Google authentication.

### What Gets Deployed

| Component | Where | Purpose |
|-----------|-------|---------|
| **Frontend + API Server** | Cloud Run | Serves the Vite-built app and handles `/api/chat` endpoints |
| **Agent** | Agent Engine | Already deployed in Step 4 — Cloud Run calls it via A2A |

### 7a. Add Your GCP Project to Firebase (One-Time Setup)

Before deploying, your GCP project must be registered with Firebase:

1. Go to the [Firebase Console](https://console.firebase.google.com/)
2. Click **"Add project"**
3. Select your existing GCP project (`a2ui-test` or your project ID)
4. Follow the prompts (you can disable Google Analytics)
5. Wait for Firebase to provision (~1 minute)

This links your GCP project to Firebase, enabling Firebase Hosting.

In [ ]:
# Enable Firebase APIs
!gcloud services enable firebase.googleapis.com firebasehosting.googleapis.com --project={PROJECT_ID} --quiet

# Install Firebase CLI silently
!npm install -g firebase-tools --registry https://registry.npmjs.org/ --silent 2>/dev/null

print("Firebase CLI installed. Now run 'firebase login' in your terminal if not already logged in.")

In [ ]:
# Verify Firebase CLI and project access
result = subprocess.run(["firebase", "--version"], capture_output=True, text=True)
if result.returncode == 0:
    print(f"✅ Firebase CLI installed: {result.stdout.strip()}")
else:
    print("❌ Firebase CLI not found. Run the cell above first.")

# Check Firebase project access
result = subprocess.run(["firebase", "projects:list"], capture_output=True, text=True)
if PROJECT_ID in result.stdout:
    print(f"✅ Firebase project '{PROJECT_ID}' is accessible")
else:
    print(f"⚠️  Project '{PROJECT_ID}' not found in Firebase. Complete Step 7a first.")

### 7b. Review Deployment Configuration

In [ ]:
# Review deployment configuration
# These variables were set earlier in the notebook

print("=" * 60)
print("DEPLOYMENT CONFIGURATION")
print("=" * 60)
print()
print(f"GCP Project ID:              {PROJECT_ID}")
print(f"GCP Project Number:          {PROJECT_NUMBER}")
print(f"Agent Engine Resource ID:    {AGENT_RESOURCE_ID}")
print(f"Region:                      {LOCATION}")
print()
print("Cloud Run Service:           personalized-learning-demo")
print(f"Firebase Hosting URL:        https://{PROJECT_ID}.web.app")
print()

# Verify all required variables are set
missing = []
if not PROJECT_ID or PROJECT_ID == "YOUR_PROJECT_ID":
    missing.append("PROJECT_ID (set in Step 2)")
if not PROJECT_NUMBER:
    missing.append("PROJECT_NUMBER (run Step 5 cell)")
if not AGENT_RESOURCE_ID or AGENT_RESOURCE_ID == "YOUR_RESOURCE_ID_HERE":
    missing.append("AGENT_RESOURCE_ID (set after Step 4)")

if missing:
    print("❌ Missing required configuration:")
    for m in missing:
        print(f"   - {m}")
else:
    print("✅ All configuration ready for deployment!")

### 7c. Deploy to Cloud Run + Firebase Hosting

This will:
1. Build and deploy the frontend + API server to Cloud Run
2. Configure Firebase Hosting to route traffic to Cloud Run
3. Set up Google authentication (users must sign in)

**First deployment takes 5-10 minutes.** Subsequent deploys are faster.

In [ ]:
# Set environment variables for the deployment script
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["AGENT_ENGINE_PROJECT_NUMBER"] = PROJECT_NUMBER
os.environ["AGENT_ENGINE_RESOURCE_ID"] = AGENT_RESOURCE_ID

# Run the deployment script
print("Starting deployment to Cloud Run + Firebase Hosting...")
print("This will take 5-10 minutes on first deploy.\n")

result = subprocess.run(
    [sys.executable, "deploy_hosting.py", 
     "--project", PROJECT_ID,
     "--region", LOCATION],
    env=os.environ
)

if result.returncode == 0:
    print("\n" + "=" * 60)
    print("🎉 DEPLOYMENT SUCCESSFUL!")
    print("=" * 60)
    print(f"\nYour demo is live at:")
    print(f"  → https://{PROJECT_ID}.web.app")
    print(f"  → https://{PROJECT_ID}.firebaseapp.com")
    print("\nShare this URL with colleagues to demo the personalized learning experience!")
else:
    print("\n❌ Deployment failed. Check the error messages above.")

### 7c-result. Verify Deployment

Run this cell to see your deployed URLs.

In [ ]:
# Get the Cloud Run URL
import subprocess

result = subprocess.run(
    ["gcloud", "run", "services", "describe", "personalized-learning-demo",
     "--region", LOCATION, "--project", PROJECT_ID, "--format", "value(status.url)"],
    capture_output=True, text=True
)
CLOUD_RUN_URL = result.stdout.strip()

print("=" * 60)
print("DEPLOYMENT COMPLETE")
print("=" * 60)
print(f"\nCloud Run URL: {CLOUD_RUN_URL}")
print(f"\nFirebase Hosting URLs:")
print(f"   https://{PROJECT_ID}.web.app")
print(f"   https://{PROJECT_ID}.firebaseapp.com")
print("\n⚠️  Complete Step 7d below to enable authentication!")

### 7d. Configure Firebase Authentication

The demo uses Firebase Authentication to restrict access to @google.com email addresses. This requires:

1. **Enable Google Sign-In provider** in Firebase Console
2. **Add Cloud Run domain** to authorized domains
3. **Make Cloud Run publicly accessible** (Firebase Auth handles the restriction)

#### Manual Steps (Firebase Console)

**Step 1: Enable Google Sign-In**

1. Go to [Firebase Console → Authentication → Sign-in method](https://console.firebase.google.com/project/a2ui-test/authentication/providers)
2. Click **"Google"** under Sign-in providers
3. Toggle **"Enable"** to ON
4. Set **"Project support email"** to your email address
5. Click **"Save"**

**Step 2: Add Authorized Domain**

1. Go to [Firebase Console → Authentication → Settings](https://console.firebase.google.com/project/a2ui-test/authentication/settings)
2. Under **"Authorized domains"**, click **"Add domain"**
3. Add your Cloud Run domain (shown below)
4. Click **"Add"**

In [ ]:
# Display the Cloud Run domain that needs to be added to Firebase authorized domains
import subprocess

result = subprocess.run(
    ["gcloud", "run", "services", "describe", "personalized-learning-demo",
     "--region", LOCATION, "--project", PROJECT_ID, "--format", "value(status.url)"],
    capture_output=True, text=True
)
CLOUD_RUN_URL = result.stdout.strip()

# Extract just the domain (remove https://)
CLOUD_RUN_DOMAIN = CLOUD_RUN_URL.replace("https://", "")

print("=" * 70)
print("ADD THIS DOMAIN TO FIREBASE AUTHORIZED DOMAINS:")
print("=" * 70)
print(f"\n   {CLOUD_RUN_DOMAIN}\n")
print("=" * 70)
print("\nSteps:")
print("1. Go to: https://console.firebase.google.com/project/" + PROJECT_ID + "/authentication/providers")
print("   → Enable Google Sign-In provider")
print("\n2. Go to: https://console.firebase.google.com/project/" + PROJECT_ID + "/authentication/settings")
print("   → Add the domain above to 'Authorized domains'")
print("\n3. Run the next cell to make Cloud Run publicly accessible")

### 7e. Verify Authentication Setup

After completing the Firebase Console steps above, your demo should be fully functional!

In [ ]:
# Final verification - print all URLs
import subprocess

result = subprocess.run(
    ["gcloud", "run", "services", "describe", "personalized-learning-demo",
     "--region", LOCATION, "--project", PROJECT_ID, "--format", "value(status.url)"],
    capture_output=True, text=True
)
CLOUD_RUN_URL = result.stdout.strip()

print("=" * 70)
print("🎉 DEPLOYMENT COMPLETE!")
print("=" * 70)
print(f"""
Your personalized learning demo is live!

URLS:
  Cloud Run:  {CLOUD_RUN_URL}
  Firebase:   https://{PROJECT_ID}.web.app

AUTHENTICATION:
  ✅ Firebase Auth restricts access to @google.com accounts
  ✅ Users see a Google Sign-In page before accessing the app

TO TEST:
  1. Open the URL above in your browser
  2. Click "Sign in with Google"
  3. Use your @google.com account
  4. Try prompts like "Help me understand ATP" or "Quiz me on bonds"

TROUBLESHOOTING:
  If you see "auth/configuration-not-found" error:
    → Enable Google Sign-In in Firebase Console (Step 7d)
    → Add Cloud Run domain to authorized domains
""")
print("=" * 70)

In [ ]:
# Step 2: Make Cloud Run publicly accessible
# Firebase Auth in the frontend handles the @google.com domain restriction
!gcloud run services add-iam-policy-binding personalized-learning-demo \
    --region={LOCATION} --project={PROJECT_ID} \
    --member="allUsers" --role="roles/run.invoker"

print("\n✅ Cloud Run service is now publicly accessible")
print("   (Firebase Auth restricts access to @google.com accounts)")

### Updating the Deployment

After making changes, redeploy with:

```bash
python deploy_hosting.py --project YOUR_PROJECT_ID
```

Or deploy only specific components:

```bash
# Redeploy only Cloud Run (frontend/API changes)
python deploy_hosting.py --cloud-run-only

# Redeploy only Firebase Hosting (routing changes)
python deploy_hosting.py --firebase-only
```

### Cleanup

To delete the deployed resources:

```bash
# Delete Cloud Run service
gcloud run services delete personalized-learning-demo --region=us-central1

# Delete Firebase Hosting site (keeps project)
firebase hosting:disable --project YOUR_PROJECT_ID
```

---

## Content Attribution

### OpenStax

Educational content is sourced from [OpenStax](https://openstax.org/), licensed under [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/).

Specifically: [Biology for AP® Courses](https://openstax.org/details/books/biology-ap-courses) - OpenStax, Rice University

---

## Security Notice

> **Warning:** When building production applications, treat any agent outside your control as potentially untrusted. This demo connects to Agent Engine within your own GCP project. Always review agent code before deploying.

---

## Limitations & Known Issues

This is a demonstration, not a production system. Here's what can/will break:

| What You Try | What Happens | Why |
|--------------|--------------|-----|
| **Ask for study materials across multiple topics at once** | Retrieval returns wrong content | The agent is designed to match to a single OpenStax chapter; multi-topic queries need more sophisticated retrieval. (There are many good ways to do this.) |
| **"Play podcast about X"** | Nothing plays (or wrong content) | Audio is pre-generated via NotebookLM, not dynamically created |
| **Sidebar navigation, settings, etc.** | Nothing happens | The UI is styled to resemble a Google product, but only the chat functionality is implemented |

### What This Demo Is (and Isn't)

**Is:** A working example of A2UI's architecture—remote agent deployment, A2A protocol, custom components, and dynamic content generation.

**Isn't:** A complete learning platform. The personalization pipeline, multi-topic retrieval, and non-chat UI elements are placeholders demonstrating where real implementations would go.